## Compress DICOMs using JPEG 2000 Lossless compression
- take a large 1.9GB DICOM multi-frame image and compress it
- HTJ2K lossless compression

In [0]:
%pip install --quiet numpy==1.26.4 cupy-cuda12x pydicom  nvidia-nvimgcodec-cu12 nvidia-nvjpeg2k-cu12 

In [0]:
dbutils.library.restartPython()

In [0]:
import pydicom
import numpy as np
import cupy as cp
from nvidia import nvimgcodec
from pydicom.encaps import encapsulate
from pydicom.uid import JPEG2000Lossless  # or JPEG2000 for lossy

# Path to your input DICOM file (uncompressed)
input_path = "/Volumes/hls_radiology/demo/random/20250227134948_CT_ISRA_0.dcm"

# Output path for the compressed DICOM file
output_path = "/Volumes/douglas_moore/demo/temp/output_jpeg2000-nv-HTJ2K.dcm"


In [0]:
ds = pydicom.dcmread(input_path)
ds.pixel_array_options(use_v2_backend=False)
pixel_array = ds.pixel_array  # shape: (frames, rows, cols)


In [0]:
# Step 1: Load the DICOM

encoded_frames = []
encoder = nvimgcodec.Encoder()
# Set HTJ2K parameters as needed; example uses lossless mode.
htj2k_params = nvimgcodec.Jpeg2kEncodeParams(
    reversible=True,
#    use_ht=True  # Some versions require 'use_ht' flag or similar for HTJ2K
    # other options if needed:
     code_block_size=(64,64),
     num_resolutions=6,
     bitstream_type=nvimgcodec.Jpeg2kBitstreamType.JP2,
     prog_order=nvimgcodec.Jpeg2kProgOrder.RPCL
)

enc_params = nvimgcodec.EncodeParams(jpeg2k_encode_params=htj2k_params)

# Step 2: For each frame, move to GPU, encode, and store results
for i in range(pixel_array.shape[0]):
    frame = pixel_array[i]  # NumPy CPU array, shape (rows, cols)
    frame_cp = cp.asarray(frame[:, :, np.newaxis])  # shape (rows, cols, 1), CuPy GPU array
    nv_img = nvimgcodec.as_image(frame_cp.astype(cp.uint16))  # pass CuPy array directly
    j2k_bytes = encoder.encode(nv_img, "jpeg2k", params=enc_params)
    encoded_frames.append(j2k_bytes)

# Step 3: Encapsulate for DICOM
ds.PixelData = encapsulate(encoded_frames)
ds['PixelData'].is_undefined_length = True

# Step 4: Set correct Transfer Syntax UID for HTJ2K (lossless)
ds.file_meta.TransferSyntaxUID = "1.2.840.10008.1.2.4.201"

ds.save_as(output_path)

In [0]:
%sh ls -lat \
/Volumes/hls_radiology/demo/random/20250227134948_CT_ISRA_0.dcm \
/Volumes/douglas_moore/demo/temp/output_jpeg2000-nv.dcm \
/Volumes/douglas_moore/demo/temp/output_jpeg2000-nv-HTJ2K.dcm